In [1]:
import numpy as np
import xarray as xr
import copy 

In [2]:
def window_stack(a, stepsize, window):
    return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


In [3]:
def r_anomaly(in_data,r_window):
    r_window_data = window_stack(in_data,1,window) #get data for running windows
    r_mean = r_window_data.mean(axis=1) #average over the each window
    
    #subtract mean to original data
    start = int((window-1)/2)
    in_data[:start]= in_data[:start]-r_mean[0] #first missing values
    in_data[-start:] = in_data[-start:]-r_mean[-1] #last missing values
    in_data[start:-start] = in_data[start:-start,:]-np.repeat(r_mean[:,np.newaxis],in_data.shape[1],axis=1) #middle values
    return(in_data)

In [4]:
stepsize = 1
window = 15
start = int((window-1)/2)

In [5]:
run = ['HIST','ssp119','ssp126','ssp245','ssp370','ssp585']

In [6]:
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
    file = 'nGE_Tmax_'+n_run+'_dm_land_europe.nc'
    
    with xr.open_dataset(path+file) as data1:
        data1 = data1.tasmax
        
    # ------------------ Load second set -----------------------
    path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/ens31_50/'
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe.nc'
    
    with xr.open_dataset(path+file) as data2:
        data2 = data2.tasmax
        
    #concatenate over ensemble dimension
    data = np.concatenate((data1.values,data2.values),axis=1)
    
    #flatten lon/lat
    data_flat = data.reshape(data.shape[0],data.shape[1],-1) 
    
    #calculate running window ensemble mean and anomalies
    r_anomaly_output = copy.deepcopy(data_flat)
    for x in range(data_flat.shape[2]):
        in_data = data_flat[:,:,x]
        r_anomaly_output[:,:,x] = r_anomaly(in_data,window)
        #print(x)
        
    r_anomaly_output = r_anomaly_output.reshape(data.shape[0],data.shape[1],data.shape[2],data.shape[3])
    
    # ------------------ save data -----------------------
    path = '/work/uo1075/u241308/data_python_PostDoc/MPI_GE_cmip6/t2max/'
    
    file = 'nGE_Tmax_'+n_run+'_dm_ens_0-30_land_europe_ens_mean_anomaly_detrend.nc'
    data1[:,:,:,:] = r_anomaly_output[:,:30,:,:]
    data1.to_netcdf(path+file)
    
    file = 'nGE_Tmax_'+n_run+'_dm_ens_31-50_land_europe_ens_mean_anomaly_detrend.nc'
    data2[:,:,:,:] = r_anomaly_output[:,30:,:,:]
    data2.to_netcdf(path+file)
    
    print(n_run + ' finished')

/tmp/ipykernel_447231/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


HIST finished


/tmp/ipykernel_447231/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


ssp119 finished


/tmp/ipykernel_447231/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


...

ssp245 finished


/tmp/ipykernel_447231/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


ssp370 finished


/tmp/ipykernel_447231/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


ssp585 finished
